# 8. Estimation 估计

假设你生活在一个有 1 万人口的小镇，想要预测即将到来的选举谁会获胜。理论上，你可以询问镇上每个人他们打算投票给谁，如果他们全都诚实回答，你就能做出可靠的预测。

但即便是在一个小镇，调查全部人口可能也不切实际。幸运的是，这并非必要。如果你随机调查一部分人，就可以利用样本来推断全体人口的投票倾向。这种利用样本来对总体进行推断的过程，被称为统计推断。

统计推断包括估计（这是本章的主题）和假设检验（这是下一章的主题）。

In [ ]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/thinkstats.py")

## 8.1 Weighting Penguins - 称量企鹅体重

假设你是在南极洲研究当地企鹅种群的研究员。你的任务之一是监测企鹅平均体重在一年中的变化情况。为环境中的每只企鹅称重是不现实的，因此你的计划是每周随机抽取 10 只企鹅，测量它们的体重，并利用样本估算整个种群的平均值——这被称为总体均值。

你可以用多种方法通过样本来估算总体均值，但我们只考虑两种：样本均值和样本中位数。它们都是合理的选择，但让我们看看哪种更好——并思考“更好”意味着什么。

为了演示目的，我们假设企鹅体重服从已知均值和标准差的正态分布，我将用 mu 和 sigma 表示这些参数，并以千克为单位赋值。

In [ ]:
mu = 3.7
sigma = 0.46

这些值是正态分布的参数，意味着它们定义了一个特定的分布。给定这些参数，我们可以使用 NumPy 模拟抽样过程，生成任意大小的样本。例如，这里是一个假设的包含 10 个体重的样本。

In [ ]:
import numpy as np

In [ ]:
np.random.seed(1)

In [ ]:
sample = np.random.normal(mu, sigma, 10)
sample

以下是样本的均值和众数。

In [ ]:
np.mean(sample), np.median(sample)

均值与众数差异显著，这促使我们思考哪个是更优的估计量。为探究此问题，我们将使用以下函数生成指定规模 n 的假设样本。

In [ ]:
def make_sample(n):
    return np.random.normal(mu, sigma, size=n)

作为首次实验，我们将观察样本均值和样本众数如何随样本量增加而变化。使用 NumPy 的 logspace 函数生成从 10 到 100,000 的 ns 序列，在对数尺度上保持等间距分布。

In [ ]:
ns = np.logspace(1, 5).astype(int)
ns[:5]

通过列表推导式，可为每个 n 值生成假设样本、计算均值并汇总结果：

In [ ]:
means = [np.mean(make_sample(n)) for n in ns]
means[:5]

对于中位数，我们也会进行同样的处理。

In [ ]:
medians = [np.median(make_sample(n)) for n in ns]
medians[:5]

像样本均值或中位数这样的统计量，用于估计总体特性，被称为估计量。

下图展示了随着样本量的增加，这些估计量的表现如何。水平线表示总体中的实际均值。

In [ ]:
from matplotlib import pyplot as plt
from thinkstats import decorate

plt.axhline(mu, color="gray", lw=1, alpha=0.5)
plt.plot(ns, means, "--", label="mean")
plt.plot(ns, medians, alpha=0.5, label="median")

decorate(xlabel="Sample size", xscale="log", ylabel="Estimate")

对于这两种估计量，随着样本量的增加，估计值会收敛到实际值。这表明它们是一致的，这是良好估计量应具备的特性之一。基于这一特性，均值和中位数似乎同样优秀。

在上图中，你可能会注意到估计值有时偏高，有时偏低——而且看起来这种变化大致围绕真实值对称分布。这启发了另一个实验：如果我们收集许多相同大小的样本并计算多个估计值，这些估计值的平均值会是多少？

以下循环通过生成 10,001 个包含 10 只企鹅的样本并计算每个样本的平均值来模拟这一场景。

In [ ]:
means = [np.mean(make_sample(n=10)) for i in range(10001)]
np.mean(means)

以下循环模拟了相同场景，但这次它计算每个样本的中位数。

In [ ]:
medians = [np.median(make_sample(n=10)) for i in range(10001)]
np.mean(medians)

这些假设中位数的平均值也非常接近实际总体均值。

这些结果表明，样本均值和样本中位数是无偏估计量，这意味着它们在平均意义上是正确的。"偏差"一词在不同语境中有不同含义，这可能会引起混淆。在此语境中，"无偏"意味着估计量的平均值等于实际值。

到目前为止，我们已经证明两种估计量都是一致且无偏的，但尚不清楚哪一种更优。让我们再做一个实验：看看哪个估计量更准确。"准确"这个词在不同语境下也有不同含义——作为一种量化方式，我们来考虑均方误差（MSE）。以下函数计算估计值与实际值之间的差异，并返回这些误差平方的平均值。


In [ ]:
def mse(estimates, actual):
    """Mean squared error of a sequence of estimates."""
    errors = np.asarray(estimates) - actual
    return np.mean(errors**2)

需要注意的是，只有在已知实际值的情况下，我们才能计算均方误差。在实际应用中，我们通常并不知道实际值——毕竟，如果我们已经知道实际值，就不需要进行估计了。但在我们的实验中，我们知道实际总体均值为 3.7 公斤，因此可以利用它来计算样本均值的均方误差。

In [ ]:
mse(means, mu)

如果我们有样本量为 10 的样本，并使用样本均值来估计总体均值，平均平方误差约为 0.021 千克平方。现在来看样本中位数的均方误差。

In [ ]:
mse(medians, mu)

如果我们使用样本中位数来估计总体均值，平均平方误差约为 0.029 千克平方。在这个例子中，样本均值优于样本中位数；一般来说，如果数据来自正态分布，样本均值是总体均值的最佳无偏估计量，因为它能最小化均方误差。

最小化均方误差是估计量应具备的良好特性，但均方误差并非总结误差的最佳方式。首先，它难以解释。在此例中，均方误差的单位是千克的平方，因此很难说明其具体含义。

一种解决方案是使用均方误差的平方根，称为“均方根误差”或 RMSE。另一种选择是使用误差绝对值的平均值，称为“平均绝对误差”或 MAE。以下函数计算一系列估计值的 MAE。

In [ ]:
def mae(estimates, actual):
    """Mean absolute error of a sequence of estimates."""
    errors = np.asarray(estimates) - actual
    return np.mean(np.abs(errors))

以下是样本均值的平均绝对误差。

In [ ]:
mae(means, mu)

以及样本中位数。

In [ ]:
mae(medians, mu)

平均而言，我们预计样本均值会偏离约 0.115 公斤，而样本中位数会偏离约 0.137 公斤。因此，至少在这个例子中，样本均值可能是更好的选择。



## 8.2 Robustness - 稳健性

现在让我们考虑另一种情况。假设有 2%的概率，当你尝试称量一只企鹅时，它意外地按下了秤上的单位按钮，导致体重以磅而非千克记录。假设这一错误未被察觉，它将在样本中引入一个异常值。

以下函数模拟了这一情景，将 2%的体重乘以转换系数 2.2 磅每千克。

In [ ]:
def make_sample_with_errors(n):
    sample = np.random.normal(mu, sigma, size=n)
    factor = np.random.choice([1, 2.2], size=n, p=[0.98, 0.02])
    return sample * factor

为了观察这对分布产生的影响，我们将生成一个大型样本。

In [ ]:
sample = make_sample_with_errors(n=1000)

为了绘制样本的分布图，我们将使用 KDE 以及第 6 章中的 Pdf 对象。

In [ ]:
from scipy.stats import gaussian_kde
from thinkstats import Pdf

kde = gaussian_kde(sample)
domain = 0, 10
pdf = Pdf(kde, domain)
pdf.plot(label="estimated density")
decorate(xlabel="Penguin weight (kg)", ylabel="Density")

除了接近 3.7 公斤的主要众数外，测量误差还引入了第二个众数，大约在 8 公斤附近。

现在让我们重复之前的实验，模拟多个样本量为 10 的样本，计算每个样本的均值，然后计算这些样本均值的平均值。

In [ ]:
means = [np.mean(make_sample_with_errors(n=10)) for i in range(10001)]
np.mean(means)

测量误差导致样本均值平均高于 3.7 公斤。

现在使用样本中位数进行同样的实验。

In [ ]:
medians = [np.median(make_sample_with_errors(n=10)) for i in range(10001)]
np.mean(medians)

样本中位数的平均值也高于 3.7 公斤，但偏差远没有那么大。如果我们比较估计值的均方误差，会发现样本中位数的准确性要高得多。

In [ ]:
mse(means, mu), mse(medians, mu)

如果测量值实际上来自正态分布，样本均值会最小化均方误差，但这种情况违反了该假设，因此样本均值不会最小化均方误差。样本中位数对异常值不那么敏感，因此偏差较小，其均方误差也较小。能够很好地处理异常值——以及类似的假设违反情况——的估计量被称为稳健估计量。

这段话的核心是在讨论**估算器（Estimator）**在不同数据环境下的“表现”和“可靠性”。作者通过比较**样本均值（Sample Mean）**和**样本中位数（Sample Median）**，揭示了统计学中一个非常重要的权衡：**数学上的完美性**与**现实中的稳健性**。

以下是这段话的详细逻辑拆解：

1. 均值的“完美”前提：正态分布假设
> “If measurements actually come from a normal distribution, the sample mean minimizes MSE...”
> （如果测量值确实来自正态分布，样本均值会使均方误差 MSE 最小化……）

*   **MSE（均方误差）**：它是衡量估算准确性的指标，计算方式是“误差的平方的平均值”。MSE 越小，代表估算越精确。
*   **数学结论**：在理想状态下（数据呈现完美的钟形正态分布），样本均值是数学上**最精确**的估算器，它的 MSE 是所有无偏估算器中最小的。

2. 现实的干扰：违反假设
> “...but this scenario violates that assumption, so the sample mean doesn’t minimize MSE.”
> （……但这个场景违反了那个假设，所以样本均值不再能使 MSE 最小化。）

*   **发生了什么？** 在 8.2 节的实验中，作者引入了**测量误差**：2% 的企鹅体重被错误地记成了磅（数值约为公斤的 2.2 倍）。
*   **结果**：数据分布不再是单一的正态分布，而是在 8 公斤附近出现了第二个“峰值（众数）”。因为前提条件（正态分布）被破坏了，样本均值也就失去了它“最精确”的宝座。

3. 中位数的优势：对异常值不敏感
> “The sample median is less sensitive to outliers, so it is less biased and its MSE is smaller.”
> （样本中位数对异常值不太敏感，因此它的偏差更小，MSE 也更小。）

*   **对异常值的反应**：均值的计算会考虑每一个数值。哪怕只有几个 8 公斤的错误数据（离群值/异常值），也会把整个均值大幅往上拉，导致估算结果偏高。而中位数只看中间位置的数，只要异常值的比例不高（比如本例中仅 2%），中位数的位置几乎不会改变。
*   **偏差（Bias）与 MSE**：在这个有误差的数据集中，均值被错误数据带跑了，导致它“在平均意义上是错的”（即产生了偏差）。相比之下，中位数的估算值更接近真实值（3.7 kg），误差更小，因此它的 MSE 也随之降低。

4. 核心概念：稳健性 (Robustness)
> “Estimators that deal well with outliers – and similar violations of assumptions – are said to be robust.”
> （那些能够很好地处理异常值以及类似违反假设情况的估算器，被称为是‘稳健’的。）

*   **稳健（Robust）的定义**：指的是一个统计方法在面对异常值、错误数据或不完全符合数学假设（如非正态分布）的情况下，依然能保持准确和有效的能力。
*   **结论**：在这个例子中，**样本中位数是一个稳健的估算器**，而样本均值则显得比较“脆弱”，容易受错误数据的误导。

5. 总结
作者想告诉你的是：**没有绝对最好的方法，只有最适合场景的方法。**
*   如果你的数据**非常干净**且符合正态分布，请用**均值**，它最精确。
*   如果你怀疑数据中存在**测量错误或异常值**（如本章的企鹅体重实验），请用**中位数**，因为它更**稳健**。

以下是对**估计器**及其**偏差**概念的详细解释：

1. 什么是估计器 (Estimator)？
**估计器**是指从样本中计算出来的、用于估计总体某个参数（属性）的统计量。

*   **核心逻辑**：由于在现实中很难对整个总体（Population）进行测量，我们会抽取一个样本（Sample），并通过计算样本的某些特征来推断总体的特征。
*   **样本均值和中位数是估计器吗？**
    **是的。** 来源明确指出，像样本均值（Sample Mean）和样本中位数（Sample Median）这样用于估计总体属性（如总体均值）的统计量，都被称为**估计器**。

2. 什么是无偏估计器 (Unbiased Estimator)？
**无偏估计器**是指在多次重复抽样的实验中，所有估计值的**平均值正好等于总体参数的真实值**。

*   **直观理解**：无偏意味着这个估计方法从长远来看是“平均正确”的，它既不会系统性地高估，也不会系统性地低估真实目标。
*   **示例**：在正态分布的假设下，通过大量模拟实验证明，**样本均值**和**样本中位数**在估计总体均值时都是**无偏的**，因为它们的多次估算平均结果非常接近设定的真实均值（如 3.7 kg）。

3. 是否还有有偏估计器 (Biased Estimator)？
**是的，统计学中确实存在有偏估计器。** 如果一个估计器的估算结果在平均意义上系统性地偏离了真实值，它就是有偏的。

来源中提供了一个非常经典的例子——**方差（Variance）的估计**：
*   **有偏估计器**：如果在计算样本方差时，分母使用样本量 **$n$**，那么这个估计器是**有偏的**。实验表明，这种方法计算出的结果平均而言会比总体的真实方差要**低**。
*   **无偏估计器**：如果在计算时将分母改为 **$n-1$**，那么得到的样本方差就是总体方差的**无偏估计器**。这种微调修正了由于样本量有限而导致的系统性偏差。

**总结：**
*   **估计器**是你的“测量工具”（如均值、中位数）。
*   **无偏**意味着你的工具“准星没歪”，多次测量的平均结果就是真实目标。
*   **有偏**意味着你的工具存在系统性误差（如计算方差时分母用 $n$ 会导致低估），需要进行数学修正（改用 $n-1$）。

这是一个非常深刻的问题。简单来说，你认为的“冲突”实际上源于对**中心极限定理 (CLT)** 的应用范围与**估计器稳健性 (Robustness)** 这两个不同概念的混淆。

作者在 8.2 章节中提出的观点与中心极限定理并不冲突，以下是详细的逻辑拆解：

1. 中心极限定理 (CLT) 到底在说什么？
中心极限定理的核心在于描述**样本均值的分布形状**，而不是均值这个指标本身是否“好用”。
*   **CLT 的结论**：如果你从任何一个具有有限均值和方差的总体中抽取 $n$ 个样本，当 $n$ 足够大时，这些**样本均值的分布（抽样分布）将趋近于正态分布**。
*   **CLT 的局限**：它并不保证均值能够抵御离群值（Outliers）的干扰。如果总体分布中存在极端错误的点（如 8.2 中提到的 2% 的单位错误），均值作为一个统计量，在计算时会给予每个点相同的权重，因此必然会被拉跑。

2. 为什么均值会受总体分布（及异常值）的影响？
你提到的“均值不应受总体分布影响”可能误解了 CLT 的含义。
*   **均值的计算特性**：均值的定义是所有观测值的总和除以样本量。这意味着即使只有 2% 的数据从 3.7kg 变成了 8kg，总和（分子）也会显著增加，从而导致均值漂移。
*   **估计器的效率与分布**：作者指出，**只有当数据确实来自正态分布时**，样本均值才是均方误差（MSE）最小的最优估计器。一旦总体分布中混入了非随机的、系统性的错误（如单位换算错误），总体就不再是单纯的正态分布，而是变成了“混合分布”，此时均值就失去了它在数学上的“最优性”。

3. 作者在 8.2 中强调的核心：稳健性 (Robustness)
作者通过企鹅体重的例子，实际上是在讨论**稳健统计 (Robust Statistics)**，这是独立于 CLT 的另一个范畴：
*   **均值的脆弱性**：均值对异常值非常敏感。在 8.2 的实验中，那 2% 的读数错误（公斤误记为磅）产生了一个大约 8kg 的“伪众数”，这直接导致样本均值的估算结果系统性偏高。
*   **中位数的稳健性**：相比之下，样本中位数只关注数据的中间位置。只要异常值的比例不到 50%，中位数就不会被极值拉走。
*   **结论**：当现实数据违反了“理想正态分布”的假设时，虽然样本均值的抽样分布可能依然符合 CLT 描述的正态形状，但它的**中心点（估算值）已经严重偏离了你想测量的真实目标**。因此，这种情况下中位数是比均值更“稳健”的估计器。

总结
*   **中心极限定理**告诉你：如果你反复抽样，你得到的很多个“均值”放在一起画个图，看起来像个钟形。
*   **作者在 8.2 中**告诉你：如果你原始数据里有脏数据（离群值），你算出来的那个“钟形”的中心点是歪的，它并不能代表真实的总体。

所以，**均值在数学分布上符合 CLT，但在性能（稳健性）上是“脆弱”的**。作者并没有否定 CLT，而是在提醒我们：在充满噪声的现实世界里，盲目依赖均值可能会导致错误的结论。

## 8.3 Estimating Variance - 估算方差

计算样本方差有两种方法的原因在于，其中一种是对总体方差的有偏估计，而另一种则是无偏估计。以下函数计算的是有偏估计量，即平方偏差之和除以 n 。

In [ ]:
def biased_var(xs):
    # Compute variance with n in the denominator
    n = len(xs)
    deviations = xs - np.mean(xs)
    return np.sum(deviations**2) / n

为了测试它，我们将模拟许多样本量为 10 的样本，计算每个样本的有偏方差，然后计算这些方差的平均值。

In [ ]:
biased_vars = [biased_var(make_sample(n=10)) for i in range(10001)]
np.mean(biased_vars)

结果约为 0.19，但在此情况下，我们知道实际总体方差约为 0.21，因此这个版本的样本方差平均而言过低——这证实了它是有偏的。

In [ ]:
actual_var = sigma**2
actual_var

以下函数计算无偏估计量，即平方偏差之和除以 n-1 。

In [ ]:
def unbiased_var(xs):
    # Compute variance with n-1 in the denominator
    n = len(xs)
    deviations = xs - np.mean(xs)
    return np.sum(deviations**2) / (n - 1)

我们可以通过生成许多样本并计算每个样本的无偏方差来测试它。

In [ ]:
unbiased_vars = [unbiased_var(make_sample(n=10)) for i in range(10001)]
np.mean(unbiased_vars)

无偏样本方差的平均值非常接近实际值——这正是我们期望无偏估计器应有的表现。

当样本量为 10 时，有偏估计与无偏估计的差异约为 10%，这可能不可忽视。当样本量为 100 时，差异仅为 1%，这个差异足够小，在实践中可能无关紧要。

In [ ]:
n = 10
1 - (n - 1) / n

In [ ]:
n = 100
1 - (n - 1) / n

## 8.4 Sampling Distribution - 抽样分布

到目前为止，我们一直在使用模拟数据，假设企鹅体重是从已知参数的正态分布中抽取的。现在让我们看看真实数据会发生什么。

2007 年至 2010 年间，南极洲帕尔默站的研究人员对当地 342 只企鹅进行了体长与体重测量。他们收集的数据可免费获取——本章节配套的笔记本中提供了下载指引。

In [ ]:
download(
    "https://raw.githubusercontent.com/allisonhorst/palmerpenguins/c19a904462482430170bfe2c718775ddb7dbb885/inst/extdata/penguins_raw.csv"
)

我们可以使用 Pandas 读取数据。

In [ ]:
import pandas as pd

penguins = pd.read_csv("penguins_raw.csv").dropna(subset=["Body Mass (g)"])
penguins.shape

该数据集包含三个企鹅物种。

In [ ]:
penguins["Species"].value_counts()

在首个示例中，我们将仅选取帽带企鹅的数据。

In [ ]:
chinstrap = penguins.query('Species.str.startswith("Chinstrap")')

我们将使用此函数绘制估计的概率密度函数。

In [ ]:
def plot_kde(sample, name="estimated density", **options):
    kde = gaussian_kde(sample)
    m, s = np.mean(sample), np.std(sample)
    plt.axvline(m, color="gray", ls=":")

    domain = m - 4 * s, m + 4 * s
    pdf = Pdf(kde, domain, name)
    pdf.plot(**options)

这是帽带企鹅体重的分布情况，单位为千克。垂直虚线表示样本均值。

In [ ]:
weights = chinstrap["Body Mass (g)"] / 1000
plot_kde(weights, "weights")
decorate(xlabel="Penguin weight (kg)", ylabel="Density")

样本均值约为 3.7 千克。

In [ ]:
sample_mean = np.mean(weights)
sample_mean

如果要求你估计总体均值，3.7 公斤是一个合理的选择——但这个估计的精确度如何呢？

回答这个问题的一种方法是计算均值的抽样分布，它显示了估计的均值在不同样本间的变化程度。如果我们知道总体中的实际均值和标准差，就可以模拟抽样过程并计算抽样分布。但如果我们已经知道总体的实际均值，就无需进行估计了！

幸运的是，有一种简单的方法可以近似抽样分布，称为重采样。其核心思想是利用样本来构建总体模型，然后使用该模型来模拟抽样过程。

更具体地说，我们将采用参数化重采样方法，即利用样本估计总体参数，再通过理论分布生成新样本。

以下函数通过正态分布实现了这一过程。请注意，新生成的样本与原始样本具有相同规模。

In [ ]:
def resample(sample):
    # Generate a sample from a normal distribution
    m, s = np.mean(sample), np.std(sample)
    return np.random.normal(m, s, len(sample))

该循环使用 resample 生成多个样本并计算每个样本的均值。

In [ ]:
sample_means = [np.mean(resample(weights)) for i in range(1001)]

下图展示了这些样本均值的分布情况。

In [ ]:
plot_kde(sample_means, "sample means")
decorate(xlabel="Sample mean of weight (kg)", ylabel="Density")

这一结果近似于样本均值的抽样分布。它展示了如果我们收集许多相同大小的样本，我们预期样本均值会有多大变化——假设我们对总体的模型是准确的。

非正式地说，我们可以看到样本均值可能低至 3.55，如果我们收集另一个相同大小的样本，也可能高达 3.9。

在第 8.4 章节中，作者使用帽带企鹅（Chinstrap penguin）的真实数据来演示**抽样分布（Sampling Distribution）**，其核心目的不是为了推翻 3.7 kg 这个均值，而是为了**量化这个估值背后的“不确定性”**。

1. 为什么要生成 1000 个样本并画图？
在现实中，你通常只有一个样本（即那 68 只企鹅的数据）。基于这一个样本，你算出了均值 3.73 kg。
作者通过**参数化重采样（Parametric Resampling）**生成 1000 个模拟样本，是为了进行一次“思想实验”：**“如果我换一批同样数量的企鹅重新测量，算出的均值会发生多大的波动？”**。

*   **画出的图（均值分布图）**：展示了这 1000 次模拟实验产生的不同均值。
*   **结果说明了什么**：它向你展示了由于“随机抽样”带来的**必然误差**。哪怕你的测量仪器百分之百精准，仅仅因为你这批抽到的企鹅和下一批不同，均值就会在这个范围内跳动。

2. 知道均值可能在 3.55 到 3.9 之间跳动，有什么意义？
这个范围告诉了你**这个估值的“分辨率”**。
*   **如果不做这个实验**：你手里只有一个冰冷的数字 3.73。你不知道它是否因为抽到了几只特别胖的企鹅才变成这样的，也不知道如果再抽一次会不会变成 3.2 或者 4.5。
*   **做了这个实验后**：你发现均值最高也就跳到 3.9，最低也就降到 3.55。这意味着，虽然 3.73 不一定绝对等于总体的真实均值，但**真实值极大概率就在这附近，且波动不会大得离谱**。

3. 这能解决“3.7 到底靠不靠谱”的问题吗？
**能，它给出了“靠谱”的数学定义。**
在统计学中，回答“靠不靠谱”不是看均值本身，而是看接下来的两个指标：

*   **标准误差（Standard Error, SE）**：这是作者在 8.5 节紧接着提出的概念。图表越宽，SE 越大，说明估值越“不靠谱”（不精确）；图表越窄，说明无论你怎么抽样结果都差不多，估值就越“靠谱”。
*   **置信区间（Confidence Interval）**：这是 8.6 节的核心。通过图表，我们可以得出 90% 的置信区间是 [3.66, 3.81] kg。这解决了你的疑问：**3.7 是靠谱的，因为我们有 90% 的把握确定真实均值就在这不到 0.2 kg 的微小波动范围内**。

总结
作者想要说明的是：**一个好的科学研究不能只给出一个“点估计值”（如 3.7），还必须给出“误差范围”（如 SE 或置信区间）**。

通过 8.4 节的模拟，作者教会了你如何使用手中的样本去推算这个“误差范围”。如果你发现均值的分布图非常宽（比如从 2kg 到 5kg），那即便算出来的均值是 3.7，你也会知道这个结果“非常不靠谱”。正是因为图中显示的范围很窄，才证明了 3.7 是一个**高质量、高精度**的估计。

## 8.5 Standard Error - 标准误差

为了量化抽样分布的宽度，一种方法是计算其标准差——这个结果被称为标准误差。

In [ ]:
standard_error = np.std(sample_means)
standard_error

在这种情况下，标准误差约为 0.045 公斤——因此，如果我们收集多个样本，预计样本均值平均会在大约 0.045 公斤的范围内波动。

人们常常混淆标准误差和标准差。请记住：

- 标准差用于量化测量值的变异程度。
- 标准误用于量化估计的精确度。

在该数据集中，帽带企鹅体重的标准差约为 0.38 千克。

In [ ]:
np.std(weights)

平均体重的标准误约为 0.046 千克。

In [ ]:
np.std(sample_means)

标准差告诉你企鹅体重差异有多大。标准误告诉你估计的精确度如何。它们回答的是不同的问题。

然而，它们之间存在关联。如果我们已知标准差和样本量，可以这样近似计算均值的标准误差：

In [ ]:
def approximate_standard_error(sample):
    n = len(sample)
    return np.std(sample) / np.sqrt(n)

In [ ]:
approximate_standard_error(weights)

这一结果与通过重采样得到的结果相近。

## 8.6 Confidence Intervals - 置信区间

另一种总结抽样分布的方法是计算置信区间。例如，一个 90%的置信区间包含了抽样分布中 90%的数值，我们可以通过计算第 5 百分位数和第 95 百分位数来找到它。以下是帽带企鹅平均体重的 90%置信区间。

In [ ]:
ci90 = np.percentile(sample_means, [5, 95])
ci90

在解释置信区间时，人们往往会说总体参数的真实值有 90%的概率落在 90%的置信区间内。在这个例子中，我们会说帽带企鹅的总体平均体重有 90%的概率介于 3.66 公斤至 3.81 公斤之间。

根据一种被称为频率主义的严格概率哲学，这种解释是不被允许的，许多统计学教材也会告诉你这种理解是错误的。

在我看来，这种禁令过于严苛。在合理的概率哲学框架下，置信区间确实符合人们的普遍理解：真实值有 90%的概率落在 90%的置信区间内。

然而，置信区间仅量化了由抽样引起的变异性——即仅测量了总体的一部分。抽样分布并未考虑其他误差来源，特别是抽样偏差和测量误差，这些将是下一节讨论的主题。